# <center>A-T-L-A-S ATLAS!</center>

### Setup:

Install required libraries -

In [611]:
%pip install numpy pandas networkx matplotlib plotly pyvis cdlib python-louvain leidenalg igraph

Note: you may need to restart the kernel to use updated packages.


Libraries -

In [612]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from cdlib import algorithms 

### Graph Creation:
create_atlas_graph() - Creates the graph based on rule "there exists an edge from A to B if you can say B after your opponent says A". A node ending with a letter x will have an outgoing edge to all nodes starting with a letter x.

In [613]:
def create_atlas_graph(data):
    """
    Creates a directed graph for the input.
    
    Parameters:
        data (list): A list of place names (countries/cities).
    
    Returns:
        G (networkx.DiGraph): A directed graph.
    """
    G = nx.DiGraph()
    
    for place in data:
        G.add_node(place)
        last_letter = place[-1].lower()  # last letter of the name, case-insensitive comparison
        for candidate in data:
            if candidate[0].lower() == last_letter:
                G.add_edge(place, candidate)
    
    return G

Read data from datasets and create graphs for each dataset.

In [614]:
with open("/Users/mago/Desktop/Projects/Atlas/data/countries.txt", "r") as file:
    countries = [line.strip() for line in file]

with open("/Users/mago/Desktop/Projects/Atlas/data/cities.txt", "r") as file:
    cities = [line.strip() for line in file]

combined = countries + cities

print(countries)
country_graph = create_atlas_graph(countries)
city_graph = create_atlas_graph(cities)
combined_graph = create_atlas_graph(combined)

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Democratic Republic of the Congo', 'Republic of the Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'The Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'I

### Graph Visualisation:

Interactive graph code, produces an interactive graph webpage -

In [615]:
from pyvis.network import Network
import random

def visualize_interactive_graph(G, title, communities=[]):
    """
    Visualizes a directed graph using Pyvis, with different colors for each detected community.

    Parameters:
        G (networkx.DiGraph): A directed graph.
        title (str): Title for the graph visualization.
        communities (list of lists): A list of detected communities (each community is a list of nodes).
    """
    net = Network(height="100%", width="100%", notebook=True, directed=True)
    
    # Generate a color for each community
    community_colors = {}
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)]) for _ in range(len(communities))]
    
    for i, community in enumerate(communities):
        for node in community:
            community_colors[node] = colors[i]  # Assign a color to each node based on its community

    # Add nodes with colors
    for node in G.nodes():
        net.add_node(node, color=community_colors.get(node, "#999999"))  # Default gray if not assigned

    # Add edges
    for edge in G.edges():
        net.add_edge(edge[0], edge[1])

    # Set visualization options
    net.set_options("""
    var options = {
      "physics": {
        "enabled": true,
        "stabilization": {
          "enabled": true,
          "iterations": 1000
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "springLength": 100,
          "springConstant": 0.08,
          "avoidOverlap": 1
        }
      }
    }
    """)

    # Show the graph
    net.show(f"{title.replace(' ', '_').lower()}.html")

In [616]:
visualize_interactive_graph(country_graph, "Country Graph")
# visualize_interactive_graph(city_graph, "City Graph")
# visualize_interactive_graph(combined_graph, "Combined Graph")

country_graph.html


Static 3D graph plotted model -

In [617]:
def visualize_static_graph(G, title):
    """
    Visualizes a static 3D graph using Networkx, Plotly and Matplotlib.
    
    Parameters:
        G (networkx.Graph): The graph to visualize.
        title (str): Title of the graph.
    """
    # Generate a 3D layout for the graph
    pos = nx.spring_layout(G, dim=3, seed=42)  # 3D spring layout
    
    # Extract node positions
    x_nodes = [pos[node][0] for node in G.nodes()]
    y_nodes = [pos[node][1] for node in G.nodes()]
    z_nodes = [pos[node][2] for node in G.nodes()]
    
    # Create edge traces
    x_edges = []
    y_edges = []
    z_edges = []
    
    for edge in G.edges():
        x_edges += [pos[edge[0]][0], pos[edge[1]][0], None]
        y_edges += [pos[edge[0]][1], pos[edge[1]][1], None]
        z_edges += [pos[edge[0]][2], pos[edge[1]][2], None]
    
    # Create node trace
    node_trace = go.Scatter3d(
        x=x_nodes, y=y_nodes, z=z_nodes,
        mode='markers',
        marker=dict(
            size=5,
            color=np.arange(len(G.nodes())),  # Color by node index
            colorscale='Viridis',  # Color scheme
            opacity=0.8
        ),
        text=list(G.nodes()),  # Node labels
        hoverinfo='text'
    )
    
    # Create edge trace
    edge_trace = go.Scatter3d(
        x=x_edges, y=y_edges, z=z_edges,
        mode='lines',
        line=dict(color='gray', width=1),
        hoverinfo='none'
    )
    
    # Create the figure
    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title=title,
            showlegend=False,
            scene=dict(
                xaxis=dict(showbackground=False),
                yaxis=dict(showbackground=False),
                zaxis=dict(showbackground=False)
            )
        )
    )
    
    # Show the figure
    fig.show()

In [618]:
# visualize_static_graph(city_graph, "City Graph - Static 3D Visualization")
visualize_static_graph(country_graph, "Country Graph - Static 3D Visualization")

### TASK 1 - Graph Analysis

#### Function for next best move based on strategies mentioned:

In [623]:
def has_dead_ends(G, node):
    """
    Checks if a given node has moves that lead to dead-end nodes.

    Parameters:
        G (networkx.DiGraph): Input graph.
        node (str): The node (place) to check.

    Returns:
        int: number of dead-ends.
    """
    possible_moves = list(G.successors(node))
    dead_end_moves = [move for move in possible_moves if G.out_degree(move) == 0]
    return len(dead_end_moves)


def best_next_move(G, current_place):
    """
    Finds the best possible move in the Atlas game based on winning strategies.

    Parameters:
        G (networkx.DiGraph): Input graph.
        current_place (str): The current place name.

    Returns:
        best_move (str): The recommended best move.
    """
    if current_place not in G:
        return "Invalid place name", []

    possible_moves = list(G.successors(current_place))

    if possible_moves == []:
        return "No valid moves (Dead End)", []

    # Strategy: If possible go to dead end so opponent loses instantly
    filtered_moves = [move for move in possible_moves if G.out_degree(move) == 0]
    
    if filtered_moves != []:
        return filtered_moves[0]

    # Strategy: If possible go to a place with the least number of outgoing dead ends
    best_move = min(possible_moves, key=lambda move: has_dead_ends(G, move))

    for move in possible_moves:
        if has_dead_ends(G, move) == has_dead_ends(G, best_move):
            # Among all moves with no outgoing dead-ends, prefer the one with the least outgoing edges to minimize opponents moves
            if has_dead_ends(G, best_move) == 0:
                if G.out_degree(move) < G.out_degree(best_move):
                    best_move = move
            # Among all moves with outgoing dead-ends, prefer the one with the most outgoing edges to minimize chances of getting trapped
            else:    
                if G.out_degree(move) > G.out_degree(best_move):
                    best_move = move

    # Strategy: If possible, we put the opponent in a well-connected position with the most possible moves (but less/no dangerous moves) such that they will not be able to trap us either -> SCCs
    if has_dead_ends(G, best_move) > 0:

        # Strategy: prefer staying in a strongly connected component so that you have high survivability chance and good future move options
        sccs = list(nx.strongly_connected_components(G))
        # return sccs
        if sccs != []:
            current_scc = next((scc for scc in sccs if current_place in scc), None)

            if current_scc:
                scc_moves = [move for move in possible_moves if move in current_scc]
                if scc_moves:
                    scc_best_move = max(scc_moves, key=lambda x: G.out_degree(x))
                    return scc_best_move
            
            else:
                # If current place is not in any SCC, go to a place in the largest SCC
                largest_scc = max(sccs, key=len)
                scc_best_move = max(largest_scc, key=lambda x: G.out_degree(x))
                return scc_best_move
        
    
    return best_move

In [620]:
import random

def random_next_move(G, current_place):
    """
    Selects a random valid move in the Atlas game.

    Parameters:
        G (networkx.DiGraph): The game graph.
        current_place (str): Current node.

    Returns:
        str: Randomly selected move.
    """
    possible_moves = list(G.successors(current_place))
    return random.choice(possible_moves) if possible_moves else "No valid moves (Dead End)"

In [644]:
import random

def simulate_game(G, start_place, strategy1, strategy2, max_turns=100):
    """
    Simulates a game of Atlas between two strategies where each place can only be played once.
    The starting strategy is randomized to ensure fairness.

    Parameters:
        G (networkx.DiGraph): The game graph.
        start_place (str): Starting place.
        strategy1 (function): First player's strategy.
        strategy2 (function): Second player's strategy.
        max_turns (int): Max moves before forced draw.

    Returns:
        str: "Optimized Strategy Wins", "Random Strategy Wins", or "Draw".
        int: Number of moves played.
    """
    place = start_place
    played_places = set()  # Track played places

    # Randomly choose which strategy goes first
    strategies = [strategy1, strategy2]
    random.shuffle(strategies)  # Shuffle so the starting strategy is random

    turn = 0  # Track turns

    while turn < max_turns:
        current_strategy = strategies[turn % 2]  # Alternate between strategies
        played_places.add(place)  # Mark current place as played

        # Find next move
        next_place = current_strategy(G, place)

        # If no valid move or if next move is already played, the other strategy wins
        if next_place == "No valid moves (Dead End)" or next_place in played_places:
            return f"{'Optimized Strategy Wins' if turn % 2 == 1 else 'Random Strategy Wins'}", turn

        # Move to the next place and continue the game
        place = next_place
        turn += 1

    return "Draw", turn  # Game ends in a draw if max moves are reached


# **Simulate multiple games**
num_games = 2000

strategy_wins = {"Optimized Strategy Wins": 0, "Random Strategy Wins": 0, "Draw": 0}
total_moves = []

wins = 0
total = 0

for _ in range(num_games):
    start = random.choice(list(country_graph.nodes()))
    result, moves = simulate_game(country_graph, start, best_next_move, random_next_move)
    total_moves.append(moves)
    strategy_wins[result] += 1
total = strategy_wins["Optimized Strategy Wins"] + strategy_wins["Random Strategy Wins"] + strategy_wins["Draw"]
wins = strategy_wins["Optimized Strategy Wins"]
print("Wins:", wins, "Total:", total)
print("win rate: " + str(wins/total))
print("Average moves before loss:", sum(total_moves) / len(total_moves))

Wins: 1027 Total: 2000
win rate: 0.5135
Average moves before loss: 4.2755


In [ ]:
# SAMPLE GRAPH TO TEST BEST MOVE FUNCTION - none of our created graphs have dead ends :(
G1 = nx.DiGraph()

places = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N"]
G1.add_nodes_from(places)

G1.add_edges_from([
    ("A", "B"), ("B", "C"), ("B", "L"), ("B", "M"),
    ("A", "D"), ("D", "E"), ("E", "F"), ("E", "C"), ("F", "A"),
    ("A", "G"), ("G", "H"), ("G", "A"),
    ("A", "I"), ("I", "J"), ("I", "N"), ("J", "K"), ("N", "M"),
])

dead_ends = [node for node in G1.nodes() if G1.out_degree(node) == 0]
print("Dead End Nodes:", dead_ends)
move = best_next_move(G1, "A")
print(move)

Dead End Nodes: ['C', 'H', 'K', 'L', 'M']
D


#### Improved Strategy:

In [659]:
def strategy_random(graph, current_node, visited):
    """Return a random valid move from the current_node."""
    moves = [n for n in graph.successors(current_node) if n not in visited]
    if moves:
        return random.choice(moves)
    return None

def strategy_improved(graph, current_node, visited):
    """
    Look ahead for every available move, count the number of moves available and minimize the moves for the opponent
    """
    moves = [n for n in graph.successors(current_node) if n not in visited]
    if not moves:
        return None
    best_move = None
    best_score = float('inf')
    for move in moves:
        new_visited = visited.union({move})
        # Count moves available for the opponent from this move
        opponent_moves = [n for n in graph.successors(move) if n not in new_visited]
        score = len(opponent_moves)
        if score < best_score:
            best_score = score
            best_move = move
    return best_move

def simulate_game(graph, start, strat1, strat2):
    """
    Simulate a game of atlas where players alternate moves.
    strat1 (improved strategy) plays as player 1, and strat2 (random strategy) plays as player 2.
    The game continues until a player cannot make a move.
    
    Returns:
        moves_sequence (list): The sequence of moves made.
        loser (int): 0 if player 1 (improved) loses, 1 if player 2 (random) loses.
    """
    visited = {start}
    current = start
    moves_sequence = [start]
    turn = 0
    while True:
        if turn % 2 == 0:
            next_move = strat1(graph, current, visited)
        else:
            next_move = strat2(graph, current, visited)
        if next_move is None:
            break  # current player loses if no valid move is available
        moves_sequence.append(next_move)
        visited.add(next_move)
        current = next_move
        turn += 1
    loser = turn % 2  # 0 if player 1 (improved) loses; 1 if player 2 (random) loses
    return moves_sequence, loser

improved_wins = 0
random_wins = 0

num_games = 1000

for i in range(num_games):
    start_country = random.choice(list(country_graph.nodes()))
    moves, loser = simulate_game(country_graph, start_country, strategy_improved, strategy_random)
    if loser == 0:
        random_wins += 1
    else:
        improved_wins += 1

print("After " + str(num_games) + " games:")
print("Improved Strategy Wins:", improved_wins)
print("Random Strategy Wins:", random_wins)
print("Win rate for improved strategy: " + str(improved_wins/num_games))

After 1000 games:
Improved Strategy Wins: 767
Random Strategy Wins: 233
Win rate for improved strategy: 0.767


In [654]:
def strategy_random(graph, current_node, visited):
    """Return a random valid move from the current_node."""
    moves = [n for n in graph.successors(current_node) if n not in visited]
    return random.choice(moves) if moves else None

def minimax(graph, current_node, visited, depth, maximizingPlayer):
    """
    Depth-limited minimax search with a simple evaluation:
      - If no moves available, return a high negative (loss) or high positive (win) value.
      - Otherwise, return the number of available moves (as a simple heuristic).
    """
    moves = [n for n in graph.successors(current_node) if n not in visited]
    if depth == 0 or not moves:
        # Terminal state: if no moves, assign a loss/win signal
        if not moves:
            return -1000 if maximizingPlayer else 1000
        return len(moves)
    
    if maximizingPlayer:
        best_value = -float('inf')
        for move in moves:
            new_visited = visited | {move}
            value = minimax(graph, move, new_visited, depth - 1, False)
            best_value = max(best_value, value)
        return best_value
    else:
        best_value = float('inf')
        for move in moves:
            new_visited = visited | {move}
            value = minimax(graph, move, new_visited, depth - 1, True)
            best_value = min(best_value, value)
        return best_value

def strategy_minimax(graph, current_node, visited, depth=3):
    """
    Use a minimax search with the specified depth to choose the best move.
    Returns the move that maximizes the minimax value.
    """
    moves = [n for n in graph.successors(current_node) if n not in visited]
    if not moves:
        return None
    best_move = None
    best_value = -float('inf')
    for move in moves:
        new_visited = visited | {move}
        value = minimax(graph, move, new_visited, depth - 1, False)
        if value > best_value:
            best_value = value
            best_move = move
    return best_move

def simulate_game(graph, start, strat1, strat2, strat1_args=None, strat2_args=None):
    """
    Simulate a game of atlas where players alternate moves.
    strat1 is used for player 1 and strat2 for player 2.
    strat1_args and strat2_args are dictionaries of extra arguments for the strategies.
    Returns the move sequence and which player lost (0 for player1, 1 for player2).
    """
    visited = {start}
    current = start
    moves_sequence = [start]
    turn = 0
    while True:
        # Get additional args or use an empty dictionary
        args = strat1_args if turn % 2 == 0 else strat2_args
        args = args if args is not None else {}
        if turn % 2 == 0:
            next_move = strat1(graph, current, visited, **args)
        else:
            next_move = strat2(graph, current, visited, **args) if 'depth' in (args or {}) else strat2(graph, current, visited)
        if next_move is None:
            break  # current player loses if no valid move available
        moves_sequence.append(next_move)
        visited.add(next_move)
        current = next_move
        turn += 1
    loser = turn % 2  # 0 if player 1 (minimax) loses, 1 if player 2 (random) loses
    return moves_sequence, loser

minimax_wins = 0
random_wins = 0
num_games = 1000
minimax_depth = 3

for i in range(num_games):
    start_country = random.choice(list(country_graph.nodes()))
    moves, loser = simulate_game(country_graph, start_country, strategy_minimax, strategy_random, strat1_args={'depth': minimax_depth}, strat2_args={})
    if loser == 0:
        random_wins += 1
    else:
        minimax_wins += 1

print("After " + str(num_games) + " games:")
print("Minimax Strategy Wins:", minimax_wins)
print("Random Strategy Wins:", random_wins)

After 1000 games:
Minimax Strategy Wins: 882
Random Strategy Wins: 118


### TASK 2 - Community Detection

#### Community Detection Algorithms:

Girvan-Newman (Edge Betweeness Centrality) -

How it works -

1. Compute the edge-betweenness centrality for each edge in the graph. Here edge-betweenness is the number of shortest paths between pairs of nodes that pass through it. This can be calculated by performing BFS for each node in the graph.

2. Remove the edge(s) with the highest betweenness score. The removed edge(s) is expected to split the graph into different communities as it has high edge-betweenness.

3. Recalculate the edge betweenness after each removal/iteration of the algorithm.

4. Repeat until graph is split into disconnected components, each of which will form a community.

In [ ]:
def detect_communities_girvan_newman(G):
    """
    Applies the Girvan-Newman algorithm to detect communities in a graph.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list): A list of sets, where each set represents a community.
    """
    clustering = algorithms.girvan_newman(G, level=1)
    communities = clustering.communities
    print("Detected ", len(communities), " communities using Girvan-Newman at depth.")
    return communities

In [ ]:
girvan_newman_country_communities = detect_communities_girvan_newman(country_graph)
print(girvan_newman_country_communities)
visualize_interactive_graph(country_graph, "Country Graph Community", girvan_newman_country_communities)

# girvan_newman_city_communities = detect_communities_girvan_newman(city_graph)
# print(girvan_newman_city_communities)
# visualize_interactive_graph(city_graph, "City Graph Community", girvan_newman_city_communities)

# girvan_newman_combined_communities = detect_communities_girvan_newman(combined_graph)
# print(girvan_newman_combined_communities)
# visualize_interactive_graph(combined_graph, "Combined Graph Community", girvan_newman_combined_communities)

Detected  2  communities using Girvan-Newman at depth.
[['Hungary', 'Armenia', 'Tunisia', 'Fiji', 'Brunei', 'Kuwait', 'Panama', 'Lebanon', 'United Kingdom', 'Palau', 'India', 'Uzbekistan', 'Guinea-Bissau', 'Mongolia', 'Cabo Verde', 'Peru', 'Ethiopia', 'Iran', 'Suriname', 'United Arab Emirates', 'Cameroon', 'China', 'Singapore', 'Myanmar', 'Dominican Republic', 'Trinidad and Tobago', 'Morocco', 'Bulgaria', 'New Zealand', 'Dominica', 'Ireland', 'Russia', 'Angola', 'Latvia', 'Bhutan', 'Switzerland', 'Comoros', 'Serbia', 'Bosnia and Herzegovina', 'Sudan', 'Estonia', 'Uruguay', 'Jordan', 'Netherlands', 'Nigeria', 'Antigua and Barbuda', 'El Salvador', 'Kazakhstan', 'San Marino', 'Mali', 'Afghanistan', 'Czech Republic', 'Egypt', 'Belarus', 'Samoa', 'Turkmenistan', 'Bangladesh', 'Equatorial Guinea', 'Nepal', 'Honduras', 'Syria', 'Tanzania', 'Saint Lucia', 'Ecuador', 'Luxembourg', 'North Korea', 'Republic of the Congo', 'Vietnam', 'Djibouti', 'East Timor', 'Zambia', 'Mexico', 'Nauru', 'Portugal

Girvan-Newman with number of communities constraint -

In [ ]:
def detect_communities_girvan_newman_with_split(G, num_splits=1):
    """
    Applies the Girvan-Newman algorithm and returns multiple levels of community detection.

    Parameters:
        G (networkx.Graph): Input graph.
        num_splits (int): The number of splits to perform.

    Returns:
        best_communities (list): List of detected communities at the chosen split.
    """
    clustering = algorithms.girvan_newman(G, level=num_splits)
    communities = clustering.communities

    print("Detected ", len(communities), " communities using Girvan-Newman at depth.")
    return communities

In [ ]:
country_split = 10
city_split = 10
combined_split = 150

girvan_newman_country_communities = detect_communities_girvan_newman_with_split(country_graph, num_splits=country_split)
print(girvan_newman_country_communities)
visualize_interactive_graph(country_graph, "Country Graph Community", girvan_newman_country_communities)

# girvan_newman_city_communities = detect_communities_girvan_newman_with_split(city_graph, num_splits=city_split)
# print(girvan_newman_city_communities)
# visualize_interactive_graph(city_graph, "City Graph Community", girvan_newman_city_communities)

# girvan_newman_combined_communities = detect_communities_girvan_newman_with_split(combined_graph, num_splits=combined_split)
# print(girvan_newman_combined_communities)
# visualize_interactive_graph(combined_graph, "Combined Graph Community", girvan_newman_combined_communities)

Detected  11  communities using Girvan-Newman at depth.
[['Armenia', 'Tunisia', 'Fiji', 'Brunei', 'Kuwait', 'Panama', 'Lebanon', 'United Kingdom', 'Palau', 'India', 'Uzbekistan', 'Guinea-Bissau', 'Mongolia', 'Cabo Verde', 'Peru', 'Ethiopia', 'Iran', 'Suriname', 'United Arab Emirates', 'Cameroon', 'China', 'Singapore', 'Myanmar', 'Dominican Republic', 'Morocco', 'Bulgaria', 'New Zealand', 'Dominica', 'Ireland', 'Russia', 'Angola', 'Latvia', 'Bhutan', 'Switzerland', 'Comoros', 'Serbia', 'Bosnia and Herzegovina', 'Sudan', 'Estonia', 'Uruguay', 'Jordan', 'Netherlands', 'Nigeria', 'Antigua and Barbuda', 'El Salvador', 'Kazakhstan', 'San Marino', 'Mali', 'Afghanistan', 'Czech Republic', 'Egypt', 'Belarus', 'Samoa', 'Turkmenistan', 'Equatorial Guinea', 'Nepal', 'Honduras', 'Syria', 'Tanzania', 'Saint Lucia', 'Ecuador', 'Luxembourg', 'North Korea', 'Vietnam', 'Republic of the Congo', 'Djibouti', 'East Timor', 'Zambia', 'Mexico', 'Nauru', 'Portugal', 'Saudi Arabia', 'Lesotho', 'Venezuela', 'Hai

Why Girvan-Newman performs poorly for these graphs -

The graph is too inter-connected - The algorithm works best when there are well-defined communities connected by a few "bridge" edges that however for the created graphs there are no clear bridge edges.

Edge-Betweenness Centrality is less effective in highly connected graphs as there are too many possible shortest paths and alternate routes which makes edge-betweenness values become more evenly distributed, making identification of max edges hard.

Why Paraguay is always chosen as the 'other' community in the country graph -
No definitive answer since there exist other countries such as Burkina Faso, Poland, Vatikan City, etc. that have the same degree as Paraguay. Since these countries have only 1 outgoing edge and no incoming edges, it is probably due to the fact that Yemen is a more popular node than the connecting nodes of Burkina Faso (Oman) or Poland (Denmark).  However, since Oman and Yemen both end in the same letter 'y', they have the same outgoing edges and hence same shortest paths from that point onward. Furthermore, Vatikan City also ends in 'y' and has the same outgoing edge as Paraguay. Considering these facts, the only possibility of Paraguay always getting selected first is the implementation of BFS and the graph itself where due to implementation Paraguay is always seen as the weakest/max betweeness node.

Louvain -

How it works -
1. All nodes are initialised as seperate communities.
2. Greedily builds communities by selecting nodes and calculating modularity with every other community. The node is added to the community with the max modularity score.
3. Process is repeated untill modularity is maxed out, i.e, no improvement in modularity.
4. A new network is built using the communities found and the algorithm is run again to optimize the community detection.

Drawbacks -
1. Louvain only works for undirected graphs.
2. Moving nodes from one community to another can cause that community to get disconnected which is why, Louvain algorithm has a tendency to discover very weekly connected communities.

In [ ]:
def detect_communities_louvain(G):
    """
    Applies the Louvain method for community detection on an undirected version of the graph.
    
    Parameters:
        G (networkx.Graph): Input graph.
    
    Returns:
        communities (list): A list of detected communities.
    """
    undirected_G = G.to_undirected()
    clustering = algorithms.louvain(undirected_G)
    communities = clustering.communities
    print("Detected ", len(communities), " communities using Louvain.")
    return communities

In [ ]:
louvain_country_communities = detect_communities_louvain(country_graph)
print(louvain_country_communities)
visualize_interactive_graph(country_graph, "Country Graph Community", louvain_country_communities)

# louvain_city_communities = detect_communities_louvain(city_graph)
# print(louvain_city_communities)
# visualize_interactive_graph(city_graph, "City Graph Community", louvain_city_communities)

# louvain_combined_communities = detect_communities_louvain(combined_graph)
# print(louvain_combined_communities)
# visualize_interactive_graph(combined_graph, "Combined Graph Community", louvain_combined_communities)

Detected  5  communities using Louvain.
[['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Malaysia', 'Malta', 'Mauritania', 'Moldova', 'Mongolia', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Latvia', 'Liberia', 'Libya', 'Lithuania', 'Bulgaria', 'Cambodia', 'Canada', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Dominica', 'Kenya', 'Romania', 'Russia', 'Rwanda', 'The Gambia', 'Tanzania', 'Tonga', 'Tunisia', 'Georgia', 'Ghana', 'Grenada', 'Guatemala', 'Guinea', 'Uganda', 'Guyana', 'Qatar', 'Jamaica', 'Federated States of Micronesia', 'Panama', 'Papua New Guinea', 'Venezuela', 'Zambia'], ['North Korea', 'Namibia', 'Nauru', 'Nepal', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Bahrain', 'Benin', 'Bhutan', 'Brazil', 'Lebanon', 'Lesotho', 'Liechtenstein', 'Luxembourg', 'Burkina Faso', 'Oman', 'Cameroon', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Taiwan'

Leiden -

How it works - Similar to Louvain
1. All nodes are initialised as separate communities.
2. Greedily builds communities by selecting nodes and calculating modularity with every other community. The node is added to the community with the max modularity score.
3. Process is repeated until modularity is maxed out, i.e, no improvement in modularity.
4. **Main difference between Louvain and Leiden** -> Once the communities are found, they are further split and merged with a randomly chosen community. This is done to increase the quality function (how well the graph is clustered) and because this randomness allows discovering the partition space in more depth.

In [ ]:
def detect_communities_leiden(G):
    """
    Applies the Leiden algorithm for community detection and returns communities.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list): A list of detected communities.
    """
    clustering = algorithms.leiden(G)
    communities = clustering.communities
    print("Detected ", len(communities), " communities using Leiden.")
    return communities

In [ ]:
leiden_country_communities = detect_communities_leiden(country_graph)
print(leiden_country_communities)
visualize_interactive_graph(country_graph, "Country Graph Community", leiden_country_communities)

# leiden_city_communities = detect_communities_leiden(city_graph)
# print(leiden_city_communities)
# visualize_interactive_graph(city_graph, "City Graph Community", leiden_city_communities)

# leiden_combined_communities = detect_communities_leiden(combined_graph)
# print(leiden_combined_communities)
# visualize_interactive_graph(combined_graph, "Combined Graph Community", leiden_combined_communities)

Detected  5  communities using Leiden.
[['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Malaysia', 'Malta', 'Mauritania', 'Moldova', 'Mongolia', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Latvia', 'Liberia', 'Libya', 'Lithuania', 'Bulgaria', 'Cambodia', 'Canada', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Kenya', 'Romania', 'Russia', 'Rwanda', 'The Gambia', 'Tanzania', 'Tonga', 'Tunisia', 'Georgia', 'Ghana', 'Grenada', 'Guatemala', 'Guinea', 'Uganda', 'Guyana', 'Qatar', 'Jamaica', 'Federated States of Micronesia', 'Panama', 'Papua New Guinea', 'Venezuela', 'Zambia'], ['North Korea', 'Namibia', 'Nauru', 'Nepal', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Bahrain', 'Benin', 'Bhutan', 'Brazil', 'Lebanon', 'Lesotho', 'Liechtenstein', 'Luxembourg', 'Iran', 'Burkina Faso', 'Oman', 'Cameroon', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Taiwan', 'Ta

Walktrap -

How it works - The principle of the algorithm is that random walks on a graph tend to get trapped into densely connected parts which form communities.
1. All nodes are initialised as seperate communities.
2. Random short walks are done on the graph and tracked.
3. The algorithm computes how similar two communities are based on how often a random walk moves between them. If some nodes are always visited together/in the same walk they are put in the same community.
4. Communities with highest similarities are merged to form larger communities.
5. The process continues until an optimal set of communities (optimal modularity) is formed.

In [ ]:
def detect_communities_walktrap(G):
    """
    Applies the walktrap algorithm for community detection on the graph.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list): A list of detected communities.
    """

    clustering = algorithms.walktrap(G)
    communities = clustering.communities

    print("Detected ", len(communities), " communities using Walktrap.")
    return communities

In [ ]:
walktrap_country_communities = detect_communities_walktrap(country_graph)
print(walktrap_country_communities)
visualize_interactive_graph(country_graph, "Country Graph Community", walktrap_country_communities)

# walktrap_city_communities = detect_communities_walktrap(city_graph)
# print(walktrap_city_communities)
# visualize_interactive_graph(city_graph, "City Graph Community", walktrap_city_communities)

# walktrap_combined_communities = detect_communities_walktrap(combined_graph)
# print(walktrap_combined_communities)
# visualize_interactive_graph(combined_graph, "Combined Graph Community", walktrap_combined_communities)

Detected  10  communities using Walktrap.
[['Afghanistan', 'North Korea', 'Namibia', 'Nicaragua', 'Nigeria', 'North Macedonia', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Malaysia', 'Malta', 'Mauritania', 'Moldova', 'Mongolia', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Latvia', 'Liberia', 'Libya', 'Lithuania', 'India', 'Bulgaria', 'Cambodia', 'Canada', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Dominica', 'Kenya', 'Romania', 'Russia', 'Rwanda', 'The Gambia', 'Tanzania', 'Tonga', 'Tunisia', 'Georgia', 'Ghana', 'Grenada', 'Guatemala', 'Guinea', 'Guyana', 'Jamaica', 'Federated States of Micronesia', 'Panama', 'Papua New Guinea', 'Venezuela', 'Zambia'], ['Nauru', 'Nepal', 'New Zealand', 'Niger', 'Norway', 'Bahrain', 'Benin', 'Bhutan', 'Brazil', 'Lebanon', 'Lesotho', 'Liechtenstein', 'Luxembourg', 'Iran', 'Oman', 'Cameroon', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Taiwan', 'Tajikistan', 

#### Analysis of communities:

In [ ]:
def find_bridge_countries_by_communities(G, communities, top_n=10):
    """
    Identify the top 'bridge' countries that connect different communities.

    Parameters:
    G (networkx.Graph or networkx.DiGraph): The country graph.
    communities (list of lists): A list where each inner list represents a detected community.
    top_n (int): The number of top bridge countries to return.

    Returns:
    list: The top 'bridge' countries connecting different communities based on betweenness centrality.
    """
    # Flatten the community structure for fast lookup
    community_map = {}
    for i, community in enumerate(communities):
        for node in community:
            community_map[node] = i  # Map each node to its community index

    # Find nodes that connect different communities
    bridge_scores = {}
    for node in G.nodes():
        node_community = community_map.get(node, -1)
        if node_community == -1:
            continue  # Skip nodes not in any community
        
        # Check if the node connects to nodes in other communities
        for neighbor in G.neighbors(node):
            neighbor_community = community_map.get(neighbor, -1)
            if neighbor_community != -1 and neighbor_community != node_community:
                bridge_scores[node] = bridge_scores.get(node, 0) + 1

    # Rank nodes by bridge score in descending order (high score = more connections to other communities)
    sorted_bridges = sorted(bridge_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Return the top N bridge countries
    bridge_countries = [node for node, _ in sorted_bridges[:top_n]]

    return bridge_countries

In [ ]:
louvain_bridge_countries = find_bridge_countries_by_communities(country_graph, louvain_country_communities)
leiden_bridge_countries = find_bridge_countries_by_communities(country_graph, leiden_country_communities)
walktrap_bridge_countries = find_bridge_countries_by_communities(country_graph, walktrap_country_communities)

print("Top bridge countries (Louvain):", louvain_bridge_countries)
print("Top bridge countries (Leiden):", leiden_bridge_countries)
print("Top bridge countries (WalkTrap):", walktrap_bridge_countries)

Top bridge countries (Louvain): ['Central African Republic', 'Czech Republic', 'Dominican Republic', 'Egypt', 'Afghanistan', 'North Korea', 'Namibia', 'Nicaragua', 'Nigeria', 'North Macedonia']
Top bridge countries (Leiden): ['Dominican Republic', 'Egypt', 'Afghanistan', 'North Korea', 'Namibia', 'Nicaragua', 'Nigeria', 'North Macedonia', 'Azerbaijan', 'South Korea']
Top bridge countries (WalkTrap): ['Egypt', 'South Korea', 'Saint Lucia', 'Samoa', 'Saudi Arabia', 'Serbia', 'Slovakia', 'Slovenia', 'Somalia', 'South Africa']


#### Modularity:

- Modularity is a metric that measures how well a graph is divided into communities. It measures how well a given partition (or clustering) of nodes into communities compares to a randomly connected network with the same degree distribution. 
- It is calculated by comparing the actual number of edges within communities to the expected number of edges in a random graph with the same degree distribution.
- A higher modularity score indicates that the detected communities are strongly connected internally and weakly connected externally. Typically, modularity values above 0.3–0.7 indicate good community structure, depending on the network.

Using networkx inbuilt function -

In [ ]:
from networkx.algorithms.community.quality import modularity

def calculate_modularity(G, communities):
    """
    Compute the modularity of a given partition of communities.

    Parameters:
    G (networkx.Graph): Input graph.
    communities (list of lists): A list where each inner list represents a community containing nodes.

    Returns:
    float: The modularity score of the given community structure.
    """
    community_set = [set(community) for community in communities]

    return modularity(G, community_set)

In [ ]:
print("Modularity score for Girvan-Newman on countries:", calculate_modularity(country_graph, girvan_newman_country_communities))
print("Modularity score for Louvain on countries:", calculate_modularity(country_graph, louvain_country_communities))
print("Modularity score for Leiden on countries:", calculate_modularity(country_graph, leiden_country_communities))
print("Modularity score for Walktrap on countries:", calculate_modularity(country_graph, walktrap_country_communities))

Modularity score for Girvan-Newman on countries: 0.0009360588677546757
Modularity score for Louvain on countries: 0.4623941369042333
Modularity score for Leiden on countries: 0.46355467940634965
Modularity score for Walktrap on countries: 0.42327762681963754


Using cdlib newman_girvan modularity function (Adjusts modularity scores for directed edges)

NOTE - negative score implies worse than random community identification in graph.

In [ ]:
from cdlib import evaluation, NodeClustering

def calculate_modularity_ng(G, communities):
    """
    Compute modularity using CDlib for directed graphs.
    
    Parameters:
    G (networkx.DiGraph): The directed graph.
    communities (list of lists): A list where each inner list represents a community containing nodes.

    Returns:
    float: The modularity score.
    """
    clustering = NodeClustering(communities, G)

    return evaluation.newman_girvan_modularity(G, clustering).score

In [ ]:
print("Modularity score for Girvan-Newman on countries (CDlib):", calculate_modularity_ng(country_graph, girvan_newman_country_communities))
print("Modularity score for Louvain on countries (CDlib):", calculate_modularity_ng(country_graph, louvain_country_communities))
print("Modularity score for Leiden on countries (CDlib):", calculate_modularity_ng(country_graph, leiden_country_communities))
print("Modularity score for Walktrap on countries (CDlib):", calculate_modularity_ng(country_graph, walktrap_country_communities))

Modularity score for Girvan-Newman on countries (CDlib): -0.4891036638428614
Modularity score for Louvain on countries (CDlib): 0.10003232280178309
Modularity score for Leiden on countries (CDlib): 0.10096944725640977
Modularity score for Walktrap on countries (CDlib): 0.06825072075112035


Using cdlib erdos renyi modularity -

In [ ]:
from cdlib import evaluation, NodeClustering

def calculate_modularity_er(G, communities):
    """
    Compute modularity using CDlib for directed graphs.
    
    Parameters:
    G (networkx.DiGraph): The directed graph.
    communities (list of lists): A list where each inner list represents a community containing nodes.

    Returns:
    float: The modularity score.
    """
    clustering = NodeClustering(communities, G)

    return evaluation.erdos_renyi_modularity(G, clustering).score

In [ ]:
print("Modularity score for Girvan-Newman on countries (CDlib):", calculate_modularity_er(country_graph, girvan_newman_country_communities))
print("Modularity score for Louvain on countries (CDlib):", calculate_modularity_er(country_graph, louvain_country_communities))
print("Modularity score for Leiden on countries (CDlib):", calculate_modularity_er(country_graph, leiden_country_communities))
print("Modularity score for Walktrap on countries (CDlib):", calculate_modularity_er(country_graph, walktrap_country_communities))

Modularity score for Girvan-Newman on countries (CDlib): 0.09666242690226004
Modularity score for Louvain on countries (CDlib): 0.5099386189062936
Modularity score for Leiden on countries (CDlib): 0.509519989916236
Modularity score for Walktrap on countries (CDlib): 0.4981925616962113


### BONUS

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

!pip install torch-geometric
!pip install scikit-learn

!pip install node2vec

zsh:1: /Users/mago/Desktop/Projects/Atlas/atlas_env/bin/pip: bad interpreter: /Users/mago/Desktop/Atlas/atlas_env/bin/python: no such file or directory
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
zsh:1: /Users/mago/Desktop/Projects/Atlas/atlas_env/bin/pip: bad interpreter: /Users/mago/Desktop/Atlas/atlas_env/bin/python: no such file or directory
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
zsh:1: /Users/mago/Desktop/Projects/Atlas/atlas_env/bin/pip: bad interpreter: /Users/mago/Desktop/Atlas/atlas_env/bin/python: no such file or directory
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
zsh:1: /Users/mago/Desktop/Projects/Atlas/atlas_env/bin/pip: bad interpreter: /Users/mago/Desktop/Atlas/atlas_env/bin/python: no such file or directory
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
zsh:1: /Users/mago/Desktop/Projects/Atlas/atlas_env/bin/pip: bad interpreter: /Users/mago/Desktop/Atlas/atlas_env/bin/python: no such fi